# Senate Roll Call Vote Scraper

## Python Setup

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import sys
from io import StringIO
import time
import math

## What Congress session would you like to scrape?

In [2]:
# What Congress session would you like to scrape?
year = 2001

## Scraper

In [3]:
# Convert year into congress session
congress_num = math.floor((year-1787)/2)
session = (year+1)%2 + 1

In [4]:
# Check out list of all Voting Records
base_url = requests.get("https://www.senate.gov/legislative/LIS/roll_call_lists/vote_menu_"
                        +str(congress_num)
                        +"_"
                        +str(session)
                        +".htm")

print(base_url.status_code)
soup = BeautifulSoup(base_url.content, 'html.parser')

nb_votes_1 = soup.find(id = "secondary_col2")
nb_votes_2 = nb_votes_1.find('table')
nb_votes_3 = nb_votes_2.findAll('td')
nb_votes_4 = nb_votes_3[0].get_text()
nb_votes = int(nb_votes_4.split('\xa0', 1)[0])
print("Number of Roll Call votes in {}: {}".format(year, nb_votes))

200
Number of Roll Call votes in 2001: 380


In [5]:
# Scrape the vote descriptions

columns = ['Vote', 'Result', 'Question', 'Issue', 'Date']
vote_desc = pd.DataFrame("", index=np.arange(nb_votes), columns=columns)

vote_desc_table = soup.findAll(class_='contenttext')
vote_desc_table

for i in range(1, nb_votes+1):
    vote_desc['Vote'][nb_votes-i]=vote_desc_table[5*i].get_text()
    vote_desc['Result'][nb_votes-i]=vote_desc_table[5*i+1].get_text()
    vote_desc['Question'][nb_votes-i]=vote_desc_table[5*i+2].get_text()
    vote_desc['Issue'][nb_votes-i]=vote_desc_table[5*i+3].get_text()
    vote_desc['Date'][nb_votes-i]=vote_desc_table[5*i+4].get_text()

vote_desc.to_csv(
    path_or_buf="../input/Vote_Description_"+str(congress_num)+"_"+str(session)+".csv"
    , sep=','
    , index = False
)

In [6]:
# Scrape each vote

for i in range(1,nb_votes+1):
    j = 0
    print(i)

    while j == 0:

        vote_num_for_url = "0"*(5-len(str(i)))+str(i)

        url = requests.get(
            "https://www.senate.gov/legislative/LIS/roll_call_lists/roll_call_vote_cfm.cfm?congress="
            +str(congress_num)
            +"&session="
            +str(session)
            +"&vote="
            +vote_num_for_url
        )
        soup = BeautifulSoup(url.content
                             , 'html.parser')

        if soup.findAll(class_='newspaperDisplay_3column') == []:
            j = 0
        else:
            j = 1

    vote_results = soup.findAll(class_='newspaperDisplay_3column')[0].findAll(class_='contenttext')[0].get_text()
    vote_results = vote_results.replace(", Giving Live Pair", "")
    vote_results_csv = StringIO(vote_results)
    vote_results = pd.read_csv(vote_results_csv, sep=",", header=None)
    vote_results.columns = ['0', vote_num_for_url]


    if i == 1:
        all_votes = vote_results
        all_votes.columns = ['Senator', vote_num_for_url]
    else:
        all_votes = all_votes.merge(vote_results, 
                                    left_on='Senator',
                                    right_on='0', 
                                    how='outer')
        for i in range(0, len(all_votes)):
            if pd.isnull(all_votes['Senator'][i]):
                all_votes['Senator'][i]=all_votes['0'][i]
        del all_votes['0']

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


 <span style="color:red">__If code is stuck on a vote, open the vote's page on your internet browser.__</span>

## Cleaning Output

In [7]:
# Replace Yea = 1 and Nay = -1 (other = 0)

for i in range(1, nb_votes+1):
    vote_num_for_url = "0"*(5-len(str(i)))+str(i)
    all_votes['Vote_'+vote_num_for_url]=0
    all_votes['Vote_'+vote_num_for_url][all_votes[vote_num_for_url]==' Yea']=1
    all_votes['Vote_'+vote_num_for_url][all_votes[vote_num_for_url]==' Nay']=-1
    del all_votes[vote_num_for_url]

/Users/nj995/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/nj995/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [8]:
all_votes['Senator_Name'] = all_votes.Senator.str[:-7].str.strip()
all_votes['Senator_Party'] = all_votes.Senator.str[-5:-4].str.strip()
all_votes['Senator_State'] = all_votes.Senator.str[-3:-1].str.strip()
del all_votes['Senator']

In [9]:
columns = ['Senator_Name', 'Senator_Party', 'Senator_State']
for i in range(1, nb_votes+1):
    vote_num_for_url = "0"*(5-len(str(i)))+str(i)
    columns.append('Vote_'+vote_num_for_url)
all_votes = all_votes[columns]

## Export

In [10]:
all_votes.to_csv(
    path_or_buf="../input/Senate_Votes_"+str(congress_num)+"_"+str(session)+".csv"
    , sep=',', index = False
)